In [32]:
import torch
import matplotlib.pyplot as plt
device = "cuda" if torch.cuda.is_available else "cpu"
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split


NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED =42 
N_SAMPLES = 1000


x_blob , y_blob  = make_blobs(
    n_samples = N_SAMPLES,
    n_features= NUM_FEATURES,
    centers = NUM_CLASSES,
    cluster_std= 1.5,
    random_state=RANDOM_SEED    
)

x_blob = torch.from_numpy(x_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)

#* train test split can still work with tensors
xtrain, xtest, ytrain, ytest = train_test_split(
    x_blob, y_blob, test_size = 0.2, random_state = RANDOM_SEED
) 

In [33]:
import torch.nn as nn
class BlobModel(nn.Module):
    def __init__(self, input_features, n_classes, n_neurons = 10) -> None:
        super().__init__()
        self.Linear_block = nn.Sequential(
            nn.Linear(input_features, n_neurons),
            nn.Linear(n_neurons, n_neurons),
            nn.Linear(n_neurons, n_classes),
        )
        
    def forward(self, x):
        return self.Linear_block(x)

## loss & optimizer


In [36]:
model = BlobModel(input_features=NUM_FEATURES, 
                    n_classes=NUM_CLASSES, 
                    ).to(device)

In [37]:
# model = blob_model(
#     n_classes= 4, 
#     input_features=2,
#     hidden_units=8
# )
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(params= model.parameters(),lr = 0.1)

In [38]:
def accuracy_fun(y_preds, y_true):
    return (torch.eq(y_preds, y_true).sum()/len(ytrain)*100).item()

In [42]:
## accuracy function
def accuracy_fn(y_true, y_pred):
    #* torch.eq => comparison between tensor, returns boolean
    correct = torch.eq(y_true,y_pred).sum().item()
    acc = correct/(len(y_pred))*100
    return acc

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)


## to make prediction for multiclasses 
- use softmax activation function
  - predict with the model
  - pass prediction to `torch.softmax()`
  - pass the result to `torch.argmax()`
  - don't forget to change the type to mac the type of the label

## TRaining


In [43]:
x_blob , y_blob  = make_blobs(
    n_samples = N_SAMPLES,
    n_features= NUM_FEATURES,
    centers = NUM_CLASSES,
    cluster_std= 1.5,
    random_state=RANDOM_SEED    
)

X_blob = torch.from_numpy(x_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)

xtrain, xtest, ytrain, ytest = train_test_split(X_blob, y_blob, test_size=0.2, random_state=RANDOM_SEED)

In [44]:

torch.manual_seed(42)
epochs = 151

xtrain , ytrain = xtrain.to(device), ytrain.to(device)
xtest, ytest = xtest.to(device), ytest.to(device)


for epoch in range(epochs):
    model.train()
    
    logits = model(xtrain)
    y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
    loss = loss_fn(logits, ytrain)
    acc = accuracy_fn(y_pred= y_preds, y_true= ytrain)
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    with torch.inference_mode():
        logits = model(xtest)
        y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
        loss_test = loss_fn(logits, ytest)
        acc_test = accuracy_fn(y_pred= y_preds, y_true= ytest)
        
        if epoch % 10 == 0:    
            print(f"Epoch: {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Acc: {acc:.2f}% | Test Loss: {loss_test.item():.4f} | Test Acc: {acc_test:.2f}%")
            


Epoch: 1/151 | Loss: 1.9090 | Acc: 25.00% | Test Loss: 1.6338 | Test Acc: 19.50%
Epoch: 11/151 | Loss: 0.7898 | Acc: 66.88% | Test Loss: 0.7733 | Test Acc: 75.50%
Epoch: 21/151 | Loss: 0.5454 | Acc: 95.25% | Test Loss: 0.5402 | Test Acc: 95.50%
Epoch: 31/151 | Loss: 0.4243 | Acc: 97.25% | Test Loss: 0.4226 | Test Acc: 97.50%
Epoch: 41/151 | Loss: 0.3498 | Acc: 97.75% | Test Loss: 0.3496 | Test Acc: 98.00%
Epoch: 51/151 | Loss: 0.2979 | Acc: 98.38% | Test Loss: 0.2982 | Test Acc: 98.50%
Epoch: 61/151 | Loss: 0.2592 | Acc: 98.88% | Test Loss: 0.2596 | Test Acc: 99.00%
Epoch: 71/151 | Loss: 0.2292 | Acc: 99.00% | Test Loss: 0.2294 | Test Acc: 99.00%
Epoch: 81/151 | Loss: 0.2052 | Acc: 99.00% | Test Loss: 0.2052 | Test Acc: 99.00%
Epoch: 91/151 | Loss: 0.1857 | Acc: 99.12% | Test Loss: 0.1853 | Test Acc: 99.00%
Epoch: 101/151 | Loss: 0.1695 | Acc: 99.12% | Test Loss: 0.1688 | Test Acc: 99.00%
Epoch: 111/151 | Loss: 0.1559 | Acc: 99.12% | Test Loss: 0.1549 | Test Acc: 99.00%
Epoch: 121/151 